## Run

### Split system in EMT, using decoupling node

In [1]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/dpsim/examples/cxx

# Define lists for timestep and prefix
delays_delay=(0, 0.0001, 0.0002, 0.0004, 0.001, 0.003, 0.01)
delays_zoh=(0.0001, 0.0002, 0.0004, 0.001, 0.003, 0.01)
delays_linear=(0.0001, 0.0002, 0.0004, 0.001, 0.003, 0.01)

prefixes_zoh=(zoh_1e-4 zoh_2e-4 zoh_4e-4 zoh_0.001 zoh_0.003 zoh_0.01)
prefixes_linear=(linear_1e-4 linear_2e-4 linear_4e-4 linear_0.001 linear_0.003 linear_0.01)
prefixes_delay=(delay_0 delay_1e-4 delay_2e-4 delay_4e-4 delay_0.001 delay_0.003 delay_0.01)

i_intfs_delay=(0 0 0 0 0 0 0)
i_intfs_zoh=(0 0 0 0 0 0)
i_intfs_linear=(0 0 0 0 0 0)

# Iterate over the lists
for i in "${!delays_delay[@]}"; do
  EMT_RC_split_decoupled_node_Ph3 --timestep=0.00005 -o delay="${delays_delay[$i]}" -o method=delay -o i-intf-0="${i_intfs_delay[$i]}" -o prefix="${prefixes_delay[$i]}"
done

for i in "${!delays_zoh[@]}"; do
  EMT_RC_split_decoupled_node_Ph3 --timestep=0.00005 -o delay="${delays_zoh[$i]}" -o method=extrapolation-zoh -o i-intf-0="${i_intfs_zoh[$i]}" -o prefix="${prefixes_zoh[$i]}"
done

for i in "${!delays_linear[@]}"; do
  EMT_RC_split_decoupled_node_Ph3 --timestep=0.00005 -o delay="${delays_linear[$i]}" -o method=extrapolation-linear -o i-intf-0="${i_intfs_linear[$i]}" -o prefix="${prefixes_linear[$i]}"
done

In [ ]:
import villas.dataprocessing.readtools as rt
import villas.dataprocessing.plottools as pt
import matplotlib.pyplot as plt
import numpy as np

# %matplotlib widget

H_v = np.array([1e-4, 2e-4, 4e-4, 0.001, 0.003, 0.01])
H_v_legends = ["1e-4", "2e-4", "4e-4", "0.001", "0.003", "0.01"]
y_1_prev_H = [0, 0, 0, 0, 0, 0]

H_v_delay = np.array([0, 1e-4, 2e-4, 4e-4, 0.001, 0.003, 0.01])
H_v_legends_delay = ["0", "1e-4", "2e-4", "4e-4", "0.001", "0.003", "0.01"]
y_1_prev_H_delay = [0, 0, 0, 0, 0, 0, 0]

results_emt = rt.read_timeseries_dpsim(
    "logs/EMT_RC_monolithic_Ph3/EMT_RC_monolithic_Ph3.csv"
)

results_emt_itm_zoh = {}
results_emt_itm_linear = {}
results_emt_itm_delay = {}

for H_v_leg in H_v_legends_delay:
    results_emt_itm_delay[H_v_leg] = rt.read_timeseries_dpsim(
        "logs/EMT_RC_split_decoupled_Ph3_delay_"
        + H_v_leg
        + "_0_0/EMT_RC_split_decoupled_Ph3_delay_"
        + H_v_leg
        + "_0_0.csv"
    )

for H_v_leg in H_v_legends:
    results_emt_itm_zoh[H_v_leg] = rt.read_timeseries_dpsim(
        "logs/EMT_RC_split_decoupled_Ph3_zoh_"
        + H_v_leg
        + "_0_0/EMT_RC_split_decoupled_Ph3_zoh_"
        + H_v_leg
        + "_0_0.csv"
    )

for H_v_leg in H_v_legends:
    results_emt_itm_linear[H_v_leg] = rt.read_timeseries_dpsim(
        "logs/EMT_RC_split_decoupled_Ph3_linear_"
        + H_v_leg
        + "_0_0/EMT_RC_split_decoupled_Ph3_linear_"
        + H_v_leg
        + "_0_0.csv"
    )

In [ ]:
pt.set_timeseries_labels(results_emt["v_2_0"], "v_2_0 - Monolithic (delay)")
pt.plot_timeseries("Co-simulation results - v_intf - delay", results_emt["v_2_0"])
pt.set_timeseries_labels(results_emt["v_2_1"], "v_2_1 - Monolithic (delay)")
pt.plot_timeseries("Co-simulation results - v_intf - delay", results_emt["v_2_1"])
pt.set_timeseries_labels(results_emt["v_2_2"], "v_2_2 - Monolithic (delay)")
pt.plot_timeseries("Co-simulation results - v_intf - delay", results_emt["v_2_2"])
plt.xlabel("t [s]")
plt.ylabel("Interface voltage")
plt.grid()

pt.set_timeseries_labels(results_emt["i_rline_0"], "i_rline_0 - Monolithic (delay)")
pt.plot_timeseries("Co-simulation results - i_intf - delay", results_emt["i_rline_0"])
pt.set_timeseries_labels(results_emt["i_rline_1"], "i_rline_1 - Monolithic (delay)")
pt.plot_timeseries("Co-simulation results - i_intf - delay", results_emt["i_rline_1"])
pt.set_timeseries_labels(results_emt["i_rline_2"], "i_rline_2 - Monolithic (delay)")
pt.plot_timeseries("Co-simulation results - i_intf - delay", results_emt["i_rline_2"])

results_att_list = results_emt_itm_delay["0"]
pt.set_timeseries_labels(results_att_list["v_intf_0"], "v_intf_0 - S_1, delay")
pt.plot_timeseries(
    "Co-simulation results - v_intf - delay", results_att_list["v_intf_0"], "--"
)
pt.set_timeseries_labels(results_att_list["v_intf_1"], "v_intf_1 - S_1, delay")
pt.plot_timeseries(
    "Co-simulation results - v_intf - delay", results_att_list["v_intf_1"], "--"
)
pt.set_timeseries_labels(results_att_list["v_intf_2"], "v_intf_2 - S_1, delay")
pt.plot_timeseries(
    "Co-simulation results - v_intf - delay", results_att_list["v_intf_2"], "--"
)

pt.set_timeseries_labels(results_att_list["i_intf_0"], "i_intf_0 - S_1, delay")
pt.plot_timeseries(
    "Co-simulation results - i_intf - delay", results_att_list["i_intf_0"], "--"
)
pt.set_timeseries_labels(results_att_list["i_intf_1"], "i_intf_1 - S_1, delay")
pt.plot_timeseries(
    "Co-simulation results - i_intf - delay", results_att_list["i_intf_1"], "--"
)
pt.set_timeseries_labels(results_att_list["i_intf_2"], "i_intf_2 - S_1, delay")
pt.plot_timeseries(
    "Co-simulation results - i_intf - delay", results_att_list["i_intf_2"], "--"
)

plt.xlabel("t [s]")
plt.ylabel("Interface current")
plt.grid()
plt.show()

In [ ]:
pt.set_timeseries_labels(results_emt["v_2_0"], "v_2_0 - Monolithic (ZOH)")
pt.plot_timeseries("Co-simulation results - v_intf - ZOH", results_emt["v_2_0"])
pt.set_timeseries_labels(results_emt["v_2_1"], "v_2_1 - Monolithic (ZOH)")
pt.plot_timeseries("Co-simulation results - v_intf - ZOH", results_emt["v_2_1"])
pt.set_timeseries_labels(results_emt["v_2_2"], "v_2_2 - Monolithic (ZOH)")
pt.plot_timeseries("Co-simulation results - v_intf - ZOH", results_emt["v_2_2"])
plt.xlabel("t [s]")
plt.ylabel("Interface voltage")
plt.grid()

pt.set_timeseries_labels(results_emt["i_rline_0"], "i_rline_0 - Monolithic")
pt.plot_timeseries("Co-simulation results - i_intf - ZOH", results_emt["i_rline_0"])
pt.set_timeseries_labels(results_emt["i_rline_1"], "i_rline_1 - Monolithic")
pt.plot_timeseries("Co-simulation results - i_intf - ZOH", results_emt["i_rline_1"])
pt.set_timeseries_labels(results_emt["i_rline_2"], "i_rline_2 - Monolithic")
pt.plot_timeseries("Co-simulation results - i_intf - ZOH", results_emt["i_rline_2"])

results_att_list = results_emt_itm_zoh["2e-4"]
pt.set_timeseries_labels(results_att_list["v_intf_0"], "v_intf_0 - S_1, ZOH")
pt.plot_timeseries(
    "Co-simulation results - v_intf - ZOH", results_att_list["v_intf_0"], "--"
)
pt.set_timeseries_labels(results_att_list["v_intf_1"], "v_intf_1 - S_1, ZOH")
pt.plot_timeseries(
    "Co-simulation results - v_intf - ZOH", results_att_list["v_intf_1"], "--"
)
pt.set_timeseries_labels(results_att_list["v_intf_2"], "v_intf_2 - S_1, ZOH")
pt.plot_timeseries(
    "Co-simulation results - v_intf - ZOH", results_att_list["v_intf_2"], "--"
)

pt.set_timeseries_labels(results_att_list["i_intf_0"], "i_intf_0 - S_1, ZOH")
pt.plot_timeseries(
    "Co-simulation results - i_intf - ZOH", results_att_list["i_intf_0"], "--"
)
pt.set_timeseries_labels(results_att_list["i_intf_1"], "i_intf_1 - S_1, ZOH")
pt.plot_timeseries(
    "Co-simulation results - i_intf - ZOH", results_att_list["i_intf_1"], "--"
)
pt.set_timeseries_labels(results_att_list["i_intf_2"], "i_intf_2 - S_1, ZOH")
pt.plot_timeseries(
    "Co-simulation results - i_intf - ZOH", results_att_list["i_intf_2"], "--"
)

plt.xlabel("t [s]")
plt.ylabel("Interface current")
plt.grid()
plt.show()

In [ ]:
pt.set_timeseries_labels(results_emt["v_2_0"], "v_2_0 - Monolithic (linear)")
pt.plot_timeseries("Co-simulation results - v_intf - linear", results_emt["v_2_0"])
pt.set_timeseries_labels(results_emt["v_2_1"], "v_2_1 - Monolithic (linear)")
pt.plot_timeseries("Co-simulation results - v_intf - linear", results_emt["v_2_1"])
pt.set_timeseries_labels(results_emt["v_2_2"], "v_2_2 - Monolithic (linear)")
pt.plot_timeseries("Co-simulation results - v_intf - linear", results_emt["v_2_2"])
plt.xlabel("t [s]")
plt.ylabel("Interface voltage")
plt.grid()

pt.set_timeseries_labels(results_emt["i_rline_0"], "i_rline_0 - Monolithic")
pt.plot_timeseries("Co-simulation results - i_intf - linear", results_emt["i_rline_0"])
pt.set_timeseries_labels(results_emt["i_rline_1"], "i_rline_1 - Monolithic")
pt.plot_timeseries("Co-simulation results - i_intf - linear", results_emt["i_rline_1"])
pt.set_timeseries_labels(results_emt["i_rline_2"], "i_rline_2 - Monolithic")
pt.plot_timeseries("Co-simulation results - i_intf - linear", results_emt["i_rline_2"])

results_att_list = results_emt_itm_linear["2e-4"]
pt.set_timeseries_labels(results_att_list["v_intf_0"], "v_intf_0 - S_1, linear")
pt.plot_timeseries(
    "Co-simulation results - v_intf - linear", results_att_list["v_intf_0"], "--"
)
pt.set_timeseries_labels(results_att_list["v_intf_1"], "v_intf_1 - S_1, linear")
pt.plot_timeseries(
    "Co-simulation results - v_intf - linear", results_att_list["v_intf_1"], "--"
)
pt.set_timeseries_labels(results_att_list["v_intf_2"], "v_intf_2 - S_1, linear")
pt.plot_timeseries(
    "Co-simulation results - v_intf - linear", results_att_list["v_intf_2"], "--"
)

pt.set_timeseries_labels(results_att_list["i_intf_0"], "i_intf_0 - S_1, linear")
pt.plot_timeseries(
    "Co-simulation results - i_intf - linear", results_att_list["i_intf_0"], "--"
)
pt.set_timeseries_labels(results_att_list["i_intf_1"], "i_intf_1 - S_1, linear")
pt.plot_timeseries(
    "Co-simulation results - i_intf - linear", results_att_list["i_intf_1"], "--"
)
pt.set_timeseries_labels(results_att_list["i_intf_2"], "i_intf_2 - S_1, linear")
pt.plot_timeseries(
    "Co-simulation results - i_intf - linear", results_att_list["i_intf_2"], "--"
)

plt.xlabel("t [s]")
plt.ylabel("Interface current")
plt.grid()
plt.show()

## Validate Results

In [ ]:
pt.set_timeseries_labels(results_emt["v_2_0"], "v_2_0 - Monolithic (delay)")
pt.plot_timeseries("Co-simulation results - v_intf_0 - delay", results_emt["v_2_0"])
plt.xlabel("t [s]")
plt.ylabel("Interface voltage")
plt.grid()

pt.set_timeseries_labels(results_emt["i_rline_0"], "i_rline_0 - Monolithic (delay)")
pt.plot_timeseries("Co-simulation results - i_intf_0 - delay", results_emt["i_rline_0"])

for k, results_att_list in results_emt_itm_delay.items():
    pt.set_timeseries_labels(results_att_list["v_intf_0"], "v_intf_0 - S_1, delay")
    pt.plot_timeseries(
        "Co-simulation results - v_intf_0 - delay", results_att_list["v_intf_0"], "--"
    )

    pt.set_timeseries_labels(results_att_list["i_intf_0"], "i_intf_0 - S_1, delay")
    pt.plot_timeseries(
        "Co-simulation results - i_intf_0 - delay", results_att_list["i_intf_0"], "--"
    )

plt.xlabel("t [s]")
plt.ylabel("Interface current")
plt.grid()
plt.show()

In [ ]:
pt.set_timeseries_labels(results_emt["v_2_0"], "v_2_0 - Monolithic (ZOH)")
pt.plot_timeseries("Co-simulation results - v_intf_0 - ZOH", results_emt["v_2_0"])
plt.xlabel("t [s]")
plt.ylabel("Interface voltage")
plt.grid()

pt.set_timeseries_labels(results_emt["i_rline_0"], "i_rline_0 - Monolithic (ZOH)")
pt.plot_timeseries("Co-simulation results - i_intf_0 - ZOH", results_emt["i_rline_0"])

for k, results_att_list in results_emt_itm_zoh.items():
    pt.set_timeseries_labels(results_att_list["v_intf_0"], "v_intf_0 - S_1, ZOH")
    pt.plot_timeseries(
        "Co-simulation results - v_intf_0 - ZOH", results_att_list["v_intf_0"], "--"
    )

    pt.set_timeseries_labels(results_att_list["i_intf_0"], "i_intf_0 - S_1, ZOH")
    pt.plot_timeseries(
        "Co-simulation results - i_intf_0 - ZOH", results_att_list["i_intf_0"], "--"
    )

plt.xlabel("t [s]")
plt.ylabel("Interface current")
plt.grid()
plt.show()

In [ ]:
pt.set_timeseries_labels(results_emt["v_2_0"], "v_2_0 - Monolithic")
pt.plot_timeseries("Co-simulation results - v_intf_0 - linear", results_emt["v_2_0"])
plt.xlabel("t [s]")
plt.ylabel("Interface voltage")
plt.grid()

pt.set_timeseries_labels(results_emt["i_rline_0"], "i_rline_0 - Monolithic")
pt.plot_timeseries(
    "Co-simulation results - i_intf_0 - linear", results_emt["i_rline_0"]
)

for k, results_att_list in results_emt_itm_linear.items():
    pt.set_timeseries_labels(results_att_list["v_intf_0"], "v_intf_0 - S_1, linear")
    pt.plot_timeseries(
        "Co-simulation results - v_intf_0 - linear", results_att_list["v_intf_0"], "--"
    )

    pt.set_timeseries_labels(results_att_list["i_intf_0"], "i_intf_0 - S_1, linear")
    pt.plot_timeseries(
        "Co-simulation results - i_intf_0 - linear", results_att_list["i_intf_0"], "--"
    )

plt.xlabel("t [s]")
plt.ylabel("Interface current")
plt.grid()
plt.show()

In [9]:
time_step = 50e-6

global_error_zoh = []
global_error_linear = []
global_error_delay = []

ts_v1_m = results_emt["v_1_0"]
ts_v2_m = results_emt["v_2_0"]
ts_i_intf_m = results_emt["i_rline_0"]

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(16, 9))

for i in range(0, len(H_v_legends)):
    H = H_v[i]

    m = int(round(H / time_step))

    ts_v1_zoh = results_emt_itm_zoh[H_v_legends[i]]["v_1_0"]
    ts_v2_zoh = results_emt_itm_zoh[H_v_legends[i]]["v_2_1_0"]
    ts_i_intf_zoh = results_emt_itm_zoh[H_v_legends[i]]["i_intf_0"]

    xy_a = np.array(
        [
            ts_v1_m.values[::m],
            ts_v2_m.values[::m],
            ts_i_intf_m.values[::m],
            ts_v2_m.values[::m],
        ]
    )
    xy_zoh = np.array(
        [
            ts_v1_zoh.values[::m],
            ts_v2_zoh.values[::m],
            ts_i_intf_zoh.values[::m],
            ts_v2_zoh.values[::m],
        ]
    )

    global_error_v1_zoh_t = np.abs(ts_v1_m.values[::m] - ts_v1_zoh.values[::m])
    global_error_v2_zoh_t = np.abs(ts_v2_m.values[::m] - ts_v2_zoh.values[::m])
    global_error_i_intf_zoh_t = np.abs(
        ts_i_intf_m.values[::m] - ts_i_intf_zoh.values[::m]
    )

    global_error_zoh_t = np.linalg.norm(xy_a - xy_zoh, axis=0)

    axes[0].plot(
        ts_v1_m.time[::m], global_error_v1_zoh_t, label="V1 - ZOH - " + H_v_legends[i]
    )
    axes[1].plot(
        ts_v2_m.time[::m], global_error_v2_zoh_t, label="V2 - ZOH - " + H_v_legends[i]
    )
    axes[2].plot(
        ts_i_intf_m.time[::m],
        global_error_i_intf_zoh_t,
        label="i_intf - ZOH - " + H_v_legends[i],
    )

    global_error_zoh.append(np.max(global_error_zoh_t))

axes[0].set_xlabel("t [s]")
axes[0].set_ylabel("Global error")
axes[0].grid()
axes[0].legend()

axes[1].set_xlabel("t [s]")
axes[1].set_ylabel("Global error")
axes[1].grid()
axes[1].legend()

axes[2].set_xlabel("t [s]")
axes[2].set_ylabel("Global error")
axes[2].grid()
axes[2].legend()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(16, 9))

for i in range(0, len(H_v_legends)):
    H = H_v[i]

    m = int(round(H / time_step))

    ts_v1_linear = results_emt_itm_linear[H_v_legends[i]]["v_1_0"]
    ts_v2_linear = results_emt_itm_linear[H_v_legends[i]]["v_2_1_0"]
    ts_i_intf_linear = results_emt_itm_linear[H_v_legends[i]]["i_intf_0"]

    xy_a = np.array(
        [
            ts_v1_m.values[::m],
            ts_v2_m.values[::m],
            ts_i_intf_m.values[::m],
            ts_v2_m.values[::m],
        ]
    )
    xy_linear = np.array(
        [
            ts_v1_linear.values[::m],
            ts_v2_linear.values[::m],
            ts_i_intf_linear.values[::m],
            ts_v2_linear.values[::m],
        ]
    )

    global_error_v1_linear_t = np.abs(ts_v1_m.values[::m] - ts_v1_linear.values[::m])
    global_error_v2_linear_t = np.abs(ts_v2_m.values[::m] - ts_v2_linear.values[::m])
    global_error_i_intf_linear_t = np.abs(
        ts_i_intf_m.values[::m] - ts_i_intf_linear.values[::m]
    )

    global_error_linear_t = np.linalg.norm(xy_a - xy_linear, axis=0)

    axes[0].plot(
        ts_v1_m.time[::m],
        global_error_v1_linear_t,
        label="V1 - linear - " + H_v_legends[i],
    )
    axes[1].plot(
        ts_v2_m.time[::m],
        global_error_v2_linear_t,
        label="V2 - linear - " + H_v_legends[i],
    )
    axes[2].plot(
        ts_i_intf_m.time[::m],
        global_error_i_intf_linear_t,
        label="i_intf - linear - " + H_v_legends[i],
    )

    global_error_linear.append(np.max(np.max(global_error_linear_t)))

axes[0].set_xlabel("t [s]")
axes[0].set_ylabel("Global error")
axes[0].grid()
axes[0].legend()

axes[1].set_xlabel("t [s]")
axes[1].set_ylabel("Global error")
axes[1].grid()
axes[1].legend()

axes[2].set_xlabel("t [s]")
axes[2].set_ylabel("Global error")
axes[2].grid()
axes[2].legend()

In [ ]:
plt.figure()
plt.plot(np.log10(H_v), np.log10(global_error_zoh), "b", label="ZOH")
plt.plot(np.log10(H_v), np.log10(global_error_zoh), "bo")
plt.plot(
    np.log10(H_v),
    np.log10(np.multiply(H_v, 10 * 0.012)),
    "k",
    label=r"$\mathcal{O}(H)$",
)
plt.plot(np.log10(H_v), np.log10(global_error_linear), "r", label="Linear")
plt.plot(np.log10(H_v), np.log10(global_error_linear), "ro")
plt.plot(
    np.log10(H_v),
    np.log10(np.multiply(H_v**2, 10**1.5)),
    "--k",
    label=r"$\mathcal{O}(H^2)$",
)
plt.xlabel("")
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()